In [2]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load preprocessed dataset with clean descriptions
df = pd.read_csv("/content/drive/MyDrive/adv DS/processed_with_features.csv")

# Drop rows with missing or blank descriptions
df = df[df['desc'].notna() & (df['desc'].str.strip() != '')]

# Use binary labels
df['label'] = df['loan_status_binary']

# Train-test split for fine-tuning
train_df, test_df = train_test_split(df[['desc', 'label']], test_size=0.2, stratify=df['label'], random_state=42)


<ipython-input-3-a9b257cbe525>:2: DtypeWarning: Columns (123,124,125,128,129,130,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/adv DS/processed_with_features.csv")
<ipython-input-3-a9b257cbe525>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['loan_status_binary']


In [ ]:
# Tokenize text
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["desc"], truncation=True)

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/11976 [00:00<?, ? examples/s]

Map:   0%|          | 0/2995 [00:00<?, ? examples/s]

In [ ]:
# Model Setup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define Trainer
training_args = TrainingArguments(
    output_dir="bert-default-model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)


<ipython-input-6-07e49cf2a375>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train Model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anushka7 (anushka7-university-of-maryland) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.656500
20,0.679500
30,0.608600
40,0.618600
50,0.593700
60,0.654700
70,0.645800
80,0.647300
90,0.683100
100,0.653700


Step,Training Loss
10,0.656500
20,0.679500
30,0.608600
40,0.618600
50,0.593700
60,0.654700
70,0.645800
80,0.647300
90,0.683100
100,0.653700


TrainOutput(global_step=2996, training_loss=0.5817802236458965, metrics={'train_runtime': 1856.2223, 'train_samples_per_second': 25.807, 'train_steps_per_second': 1.614, 'total_flos': 5099047030664160.0, 'train_loss': 0.5817802236458965, 'epoch': 4.0})

In [ ]:
# Load structured dataset
df = pd.read_csv("/content/drive/MyDrive/adv DS/processed_with_features.csv")

# Filter out missing or blank descriptions
df = df[df['desc'].notna() & (df['desc'].str.strip() != '')].copy()
df = df.reset_index(drop=True)

print(f"Data loaded for scoring: {df.shape}")

Data loaded for scoring: (14971, 149)


<ipython-input-8-15374ea1ff9e>:2: DtypeWarning: Columns (123,124,125,128,129,130,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/adv DS/processed_with_features.csv")


In [ ]:
# Load the same tokenizer used while training
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["desc"], truncation=True)

# Convert pandas → HuggingFace Dataset → tokenize
full_ds = Dataset.from_pandas(df[['desc']])
full_ds = full_ds.map(tokenize, batched=True)


Map:   0%|          | 0/14971 [00:00<?, ? examples/s]

In [ ]:
# Predict with fine-tuned BERT model
preds = trainer.predict(full_ds)

# Get probability of class 1 (Charged Off = default)
risk_scores = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=1)[:, 1].numpy()

In [ ]:
# Add BERT risk score to the original DataFrame
df['bert_risk_score'] = risk_scores

# Save for use in traditional models
df.to_csv("processed_with_bert_score.csv", index=False)
print("Saved: processed_with_bert_score.csv with BERT-based risk scores.")

✅ Saved: processed_with_bert_score.csv with BERT-based risk scores.


In [15]:
df[['desc', 'loan_status_binary', 'bert_risk_score']].head(5)

,desc,loan_status_binary,bert_risk_score
0,Borrower added on 07/08/13 > This loan is to...,0,0.023886
1,Borrower added on 07/06/12 > debt relief<br>...,0,0.332330
2,Borrower added on 05/01/12 > I am applying f...,0,0.023767
3,Borrower added on 02/04/13 > consolidate cre...,1,0.756039
4,Borrower added on 10/29/13 > I am consolidat...,0,0.650122
